In [1]:
import os

import pymongo
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.collection import Collection
from tqdm.auto import tqdm

load_dotenv("../env")
client = MongoClient(os.environ.get("MONGO_DB_CONNECTION"))
db = client.get_database("prismai")

In [5]:
"""
CHEAT Ghostbuster HC3 MAGE OpenLLMText SeqXGPT
"""

'\nCHEAT Ghostbuster HC3 MAGE OpenLLMText SeqXGPT\n'

In [3]:
collected_items = db.get_collection("collected_items")
synthesized_texts = db.get_collection("synthesized_texts")
features_prismai = db.get_collection("features_prismai")
test_swt = db.get_collection("test_swt")
test_new = db.get_collection("test_new")
dataset_CHEAT = db.get_collection("dataset_CHEAT")

In [ ]:
# for ds in "CHEAT Ghostbuster HC3 MAGE OpenLLMText SeqXGPT".split():
#     for index in tqdm(list(features_prismai.list_indexes())):
#         db.get_collection(f"features_{ds}").create_index(index.get("key"))

100%|██████████| 7/7 [00:00<00:00, 142.11it/s]


In [123]:
from transformers import AutoTokenizer


def normalize_text(text: str):
    return " ".join(text.strip().split())


def get_synth_chunk_range(
    tokenizer: AutoTokenizer,
    doc: dict,
    synth: dict,
):
    full_text = normalize_text(synth["text"])
    encoding = tokenizer(
        full_text,
        return_offsets_mapping=True,
        return_special_tokens_mask=True,
        add_special_tokens=True,
        truncation=False,
    )
    synth_text: str = normalize_text(synth["synth_metadata"]["synth_text"])

    if not synth_text:
        return None

    chunk_start_idx = synth["synth_metadata"]["start_idx"]
    prefix = " ".join(synth["chunks"][:chunk_start_idx])
    prefix = normalize_text(prefix)

    char_start_index = full_text.index(synth_text, len(prefix))
    token_start_idx = encoding.char_to_token(char_start_index)

    # synth_text_is_first_text = chunk_start_idx == 0 or not prefix

    if token_start_idx is None:
        raise ValueError(f"Token start index is None: {synth['_id']}")

    synth_encoding = tokenizer(
        " " + synth_text
        if "gpt" in tokenizer.name_or_path and char_start_index > 0
        else synth_text,
        add_special_tokens=False,
        return_length=True,
        truncation=False,
    )
    (synth_length,) = synth_encoding["length"]
    token_end_idx = token_start_idx + synth_length

    # sanity check
    decoded_synth_text = tokenizer.decode(
        encoding.input_ids[token_start_idx:token_end_idx],
        clean_up_tokenization_spaces=False,
    )
    if synth_text.strip() != decoded_synth_text.strip():
        raise ValueError(
            f"Decoding mismatch: {synth['_id']}!\n  '{synth_text}'\n!='{decoded_synth_text}'\n"
        )

    return doc["_id"], token_start_idx, token_end_idx
    # else:
    #     synth_chunks_char_idx = full_text.index(first_synth_chunk)
    #     token_start_idx = encoding.char_to_token(synth_chunks_char_idx)
    #     if first_synth_chunk in full_text[: synth_chunks_char_idx + 1]:
    #         raise ValueError("Chunk not unique in text!")

In [126]:
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model, use_fast=True
)
num_docs = features_prismai.count_documents(
    {"document.type": "chunk", "model.name": model},
)
failures = []
batch_size = 1000
with (
    tqdm(total=num_docs, position=0, leave=True) as tq_find,
    tqdm(total=batch_size, position=1, leave=True) as tq_update,
):
    for skip in range(0, num_docs, batch_size):
        results = []
        for doc in features_prismai.find(
            {"document.type": "chunk", "model.name": model},
            projection=["document._synth_id.$id"],
            skip=skip,
            limit=batch_size,
        ):
            synth = synthesized_texts.find_one(
                {"_id": doc["document"]["_synth_id"]["$id"]}
            )
            try:
                result = get_synth_chunk_range(tokenizer, doc, synth)
                if result is not None:
                    results.append(result)
            except ValueError:
                failures.append(doc["_id"])
                tq_find.set_postfix_str(f"failures: {len(failures)}")
            tq_find.update(1)

        tq_update.reset(len(results))
        for _id, start, end in results:
            features_prismai.update_one(
                {"_id": _id},
                {
                    "$set": {
                        "document.synth_token_start": start,
                        "document.synth_token_end": end,
                    }
                },
            )
            tq_update.update(1)

100%|██████████| 22635/22635 [11:15<00:00, 33.49it/s]


{'_id': ObjectId('6797bddabb95bc200c6ad37a'),
 'ref_id': DBRef('collected_items', 'bd314092-bcff-4adc-b372-8a3d1d9b23c9'),
 '_ref_id': DBRef('collected_items', 'ec3f8da5-6dab-4488-af89-3815f5fa6d0d'),
 'domain': 'student_essays',
 'date': '-',
 'source': 'https://www.kaggle.com/datasets/thedrcat/daigt-proper-train-dataset',
 'lang': 'en-EN',
 'agent': 'gemma2:9b',
 'text': 'In the passage "Driverless Cars Are Coming," the author gives reasons why the car would be nice to have, but they also give reasons why it would not be good to have. Even though having a car drive you around would be nice, to me the benefits of driving yourself around are so much more than thoughs of having a car drive you around. Having a car driving you around all the time can create many dangers to the people around you, possible malfunctions can occur to the car, and the joy of driving your own car around would be lost.  The dangers of having a car drive you around would be that the human brain is marter than a 

In [ ]:
raise RuntimeError("STOP")

In [1]:
class Averager:
    def __init__(self):
        self.sum = 0
        self.count = 0

    def update(self, value):
        self.sum += value
        self.count += 1

    def average(self):
        return self.sum / self.count

In [3]:
from collections import defaultdict

import numpy as np
import pandas as pd

In [4]:
from transformers import AutoTokenizer

gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", use_fast=True)

In [49]:
prismai_gpt2 = defaultdict(list)
prismai_llama = defaultdict(list)

In [50]:
for domain in tqdm(db.get_collection("collected_items").distinct("domain"), position=0):
    for item in tqdm(
        db.get_collection("synthesized_texts").find(
            {"domain": domain},
            projection={"text": 1},
        ),
        position=1,
    ):
        text = " ".join(item.get("text").strip().split())
        encoding = gpt_tokenizer(text, return_length=True)
        (length,) = encoding["length"]
        prismai_gpt2[domain].append(length)

        encoding = llama_tokenizer(text, return_length=True)
        (length,) = encoding["length"]
        prismai_llama[domain].append(length)

  0%|          | 0/9 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [66]:
for domain in tqdm(db.get_collection("collected_items").distinct("domain"), position=0):
    for item in tqdm(
        db.get_collection("features_prismai").aggregate(
            [
                {"$match": {"document.domain": domain}},
                {"$project": {"_id": 1, "document._id.$id": 1}},
                {
                    "$lookup": {
                        "from": "collected_items",
                        "localField": "document._id.$id",
                        "foreignField": "_id",
                        "as": "text",
                        "pipeline": [
                            {"$project": {"text": 1}},
                        ],
                    }
                },
                {"$unwind": "$text"},
                {"$project": {"text": "$text.text"}},
            ]
        ),
        desc=domain,
        position=1,
    ):
        text = " ".join(item.get("text").strip().split())
        encoding = gpt_tokenizer(text, return_length=True)
        (length,) = encoding["length"]
        prismai_gpt2[domain].append(length)

        encoding = llama_tokenizer(text, return_length=True)
        (length,) = encoding["length"]
        prismai_llama[domain].append(length)

  0%|          | 0/9 [00:00<?, ?it/s]

arxiv_papers: 0it [00:00, ?it/s]

blog_authorship_corpus: 0it [00:00, ?it/s]

bundestag: 0it [00:00, ?it/s]

cnn_news: 0it [00:00, ?it/s]

euro_court_cases: 0it [00:00, ?it/s]

gutenberg: 0it [00:00, ?it/s]

house_of_commons: 0it [00:00, ?it/s]

spiegel_articles: 0it [00:00, ?it/s]

student_essays: 0it [00:00, ?it/s]

In [69]:
df = pd.DataFrame(
    [{"gpt2": prismai_gpt2[key], "llama": prismai_llama[key]} for key in prismai_llama],
    columns=["gpt2", "llama"],
    index=list(prismai_llama.keys()),
)
df["gpt2_05p"] = df["gpt2"].map(lambda x: np.percentile(x, 5))
# df["gpt2_mean"] = df["gpt2"].map(np.mean)
df["gpt2_median"] = df["gpt2"].map(np.median)
df["gpt2_95p"] = df["gpt2"].map(lambda x: np.percentile(x, 95))
del df["gpt2"]
df["llama_05p"] = df["llama"].map(lambda x: np.percentile(x, 5))
# df["llama_mean"] = df["llama"].map(np.mean)
df["llama_median"] = df["llama"].map(np.median)
df["llama_95p"] = df["llama"].map(lambda x: np.percentile(x, 95))
del df["llama"]
print(df.to_latex(float_format="\\np{%d}"))
df

\begin{tabular}{lrrrrrr}
\toprule
 & gpt2_05p & gpt2_median & gpt2_95p & llama_05p & llama_median & llama_95p \\
\midrule
arxiv_papers & \np{1009} & \np{11338} & \np{34941} & \np{966} & \np{11158} & \np{34433} \\
blog_authorship_corpus & \np{15} & \np{65} & \np{600} & \np{16} & \np{66} & \np{593} \\
bundestag & \np{234} & \np{1342} & \np{2483} & \np{170} & \np{946} & \np{1747} \\
cnn_news & \np{309} & \np{749} & \np{1597} & \np{309} & \np{748} & \np{1588} \\
euro_court_cases & \np{258} & \np{984} & \np{5046} & \np{280} & \np{1019} & \np{5140} \\
gutenberg & \np{784} & \np{39006} & \np{222774} & \np{778} & \np{37531} & \np{202753} \\
house_of_commons & \np{89} & \np{818} & \np{18497} & \np{91} & \np{822} & \np{18700} \\
spiegel_articles & \np{334} & \np{912} & \np{2603} & \np{250} & \np{682} & \np{1934} \\
student_essays & \np{213} & \np{439} & \np{890} & \np{212} & \np{436} & \np{884} \\
\bottomrule
\end{tabular}



,gpt2_05p,gpt2_median,gpt2_95p,llama_05p,llama_median,llama_95p
arxiv_papers,1009.00,11338.0,34941.5,966.50,11158.0,34433.00
blog_authorship_corpus,15.00,65.0,600.0,16.00,66.0,593.00
bundestag,234.00,1342.0,2483.9,170.00,946.0,1747.00
cnn_news,309.00,749.0,1597.0,309.55,748.0,1588.00
euro_court_cases,258.25,984.0,5046.0,280.25,1019.0,5140.00
gutenberg,784.90,39006.0,222774.0,778.90,37531.0,202753.00
house_of_commons,89.00,818.0,18497.4,91.00,822.0,18700.55
spiegel_articles,334.00,912.0,2603.0,250.00,682.0,1934.00
student_essays,213.00,439.0,890.0,212.00,436.0,884.00


In [8]:
values_gpt2 = defaultdict(list)
values_llama = defaultdict(list)

for ds in "CHEAT Ghostbuster HC3-Plus MAGE OpenLLMText SeqXGPT".split():
    for item in tqdm(db.get_collection(f"dataset_{ds}").find(), position=0, desc=ds):
        text = " ".join(item.get("text").strip().split())
        encoding = gpt_tokenizer(text, return_length=True)
        (length,) = encoding["length"]
        values_gpt2[ds].append(length)

        encoding = llama_tokenizer(text, return_length=True)
        (length,) = encoding["length"]
        values_llama[ds].append(length)

CHEAT: 0it [00:00, ?it/s]

Ghostbuster: 0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1028 > 1024). Running this sequence through the model will result in indexing errors


HC3-Plus: 0it [00:00, ?it/s]

MAGE: 0it [00:00, ?it/s]

OpenLLMText: 0it [00:00, ?it/s]

SeqXGPT: 0it [00:00, ?it/s]

In [68]:
df = pd.DataFrame(
    [{"gpt2": values_gpt2[key], "llama": values_llama[key]} for key in values_llama],
    columns=["gpt2", "llama"],
    index=list(values_llama.keys()),
)
df["gpt2_05p"] = df["gpt2"].map(lambda x: np.percentile(x, 5))
# df["gpt2_mean"] = df["gpt2"].map(np.mean)
df["gpt2_median"] = df["gpt2"].map(np.median)
df["gpt2_95p"] = df["gpt2"].map(lambda x: np.percentile(x, 95))
del df["gpt2"]
df["llama_05p"] = df["llama"].map(lambda x: np.percentile(x, 5))
# df["llama_mean"] = df["llama"].map(np.mean)
df["llama_median"] = df["llama"].map(np.median)
df["llama_95p"] = df["llama"].map(lambda x: np.percentile(x, 95))
del df["llama"]
print(df.to_latex(float_format="\\np{%d}"))
df

\begin{tabular}{lrrrrrr}
\toprule
 & gpt2_05p & gpt2_median & gpt2_95p & llama_05p & llama_median & llama_95p \\
\midrule
CHEAT & \np{106} & \np{176} & \np{298} & \np{105} & \np{173} & \np{291} \\
Ghostbuster & \np{280} & \np{632} & \np{997} & \np{281} & \np{631} & \np{998} \\
HC3-Plus & \np{12} & \np{52} & \np{383} & \np{11} & \np{41} & \np{257} \\
MAGE & \np{36} & \np{141} & \np{951} & \np{37} & \np{142} & \np{952} \\
OpenLLMText & \np{120} & \np{392} & \np{1024} & \np{120} & \np{390} & \np{1031} \\
SeqXGPT & \np{72} & \np{270} & \np{504} & \np{73} & \np{270} & \np{499} \\
\bottomrule
\end{tabular}



,gpt2_05p,gpt2_median,gpt2_95p,llama_05p,llama_median,llama_95p
CHEAT,106.0,176.0,298.0,105.0,173.0,291.00
Ghostbuster,280.0,632.0,997.0,281.0,631.0,998.05
HC3-Plus,12.0,52.0,383.0,11.0,41.0,257.00
MAGE,36.0,141.0,951.0,37.0,142.0,952.00
OpenLLMText,120.0,392.0,1024.0,120.0,390.0,1031.00
SeqXGPT,72.0,270.0,504.0,73.0,270.0,499.00


In [ ]:
list(
    features_prismai.aggregate(
        [
            {
                "$project": {
                    "domain": "$document.domain",
                    "size": {"$size": "$transition_scores.intermediate_logits"},
                },
            },
            {
                "$group": {
                    "_id": "$domain",
                    "size": {"$max": "$size"},
                },
            },
        ]
    )
)

In [ ]:
source_collection = collected_items
source_collection_limit = 1500

synth_collection = "synthesized_texts"
score_collection = "features_prismai"

domain = "cnn_news"
lang = "en-EN"

synth_type = "fulltext"
synth_agent = "gpt-4o-mini"

feature_model = "gpt2"

pre_processor_type = "truncated"

next(
    db.get_collection(score_collection).aggregate(
        [
            {
                "$match": {
                    "document.agent": {"$in": [None, synth_agent]},
                    "document.domain": domain,
                    "document.lang": lang,
                    "document.type": {"$in": ["source", synth_type]},
                    "model.name": feature_model,
                    # "pre_processor.type": pre_processor_type,
                }
            },
            {
                "$group": {
                    "_id": "$document._id.$id",
                    "count": {"$count": {}},
                    "features": {
                        "$push": {
                            "type": "$document.type",
                            "split": "$split",
                            "transition_scores": "$transition_scores",
                        }
                    },
                }
            },
        ],
        allowDiskUse=True,
    )
)

In [ ]:
features_prismai.count_documents({"model.name": {"$ne": "gpt2"}})

In [ ]:
features_prismai.delete_many({"model.name": {"$ne": "gpt2"}})

In [ ]:
list(
    features_prismai.aggregate(
        [
            {"$match": {"model.name": "meta-llama/Llama-3.2-1B"}},
            {
                "$group": {
                    "_id": ["$document.agent", "$document.type", "$document.domain"],
                    "count": {"$count": {}},
                }
            },
        ]
    )
)

In [ ]:
raise RuntimeError

In [ ]:
synthesized_texts.count_documents(
    {
        "domain": "blog_authorship_corpus",
    }
)

In [ ]:
count_ = list(
    synthesized_texts.aggregate(
        [
            {"$match": {"agent": {"$ne": "nemotron"}}},
            {
                "$group": {
                    "_id": ["$domain", "$type", "$agent"],
                    "agent": {"$first": "$agent"},
                    "domain": {"$first": "$domain"},
                    "type": {"$first": "$type"},
                    "count": {"$count": {}},
                }
            },
            {"$group": {"_id": ["$domain"], "count": {"$min": "$count"}}},
        ]
    )
)
count_

In [ ]:
items = list(
    collected_items.aggregate(
        [
            {
                "$lookup": {
                    "from": "synthesized_texts",
                    "as": "synthesized_texts",
                    "localField": "_id",
                    "foreignField": "_ref_id.$id",
                    "pipeline": [
                        {
                            "$project": {
                                "_id": 1,
                                "type": 1,
                                "agent": 1,
                                # "_doc_id": "$_ref_id.$id",
                                # "domain": 1,
                            }
                        },
                        {"$match": {"type": "fulltext", "agent": "gemma2:9b"}},
                    ],
                }
            },
        ]
    )
)
# len(items)
items

In [ ]:
# db.drop_collection(transition_scores)

In [ ]:
hoc_ids = [
    doc["_id"]
    for doc in collected_items.find(
        {"domain": "house_of_commons"}, projection={"_id": 1}, limit=1500
    )
]
len(hoc_ids)

In [ ]:
llama_ids_hoc = [
    doc["document"]["_id"].id
    for doc in transition_scores.find(
        {
            "document.domain": "house_of_commons",
            "document.type": "source",
            # "model.name": "gpt2",
            "model.name": "meta-llama/Llama-3.2-1B",
        },
        projection={"document._id": 1},
    )
]
len(llama_ids_hoc)

In [ ]:
len(set(llama_ids_hoc).intersection(hoc_ids))

In [ ]:
len(set(llama_ids_hoc).intersection(hoc_ids[: len(llama_ids_hoc)]))

In [ ]:
1500 - len(llama_ids_hoc)

In [ ]:
hoc_ids_synth = [
    doc["_id"]
    for doc in synthesized_texts.find(
        # {"domain": "euro_court_cases"},
        {"domain": "house_of_commons"},
        projection={"_id": 1},
    )
]
len(hoc_ids_synth)

In [ ]:
llama_ids_hoc_synth = [
    doc["document"]["_synth_id"].id
    for doc in transition_scores.find(
        {
            "document.domain": "house_of_commons",
            "document.type": {"$ne": "source"},
            "model.name": "meta-llama/Llama-3.2-1B",
        },
        projection={"document._synth_id": 1},
    )
]
len(llama_ids_hoc)

In [ ]:
len(set(llama_ids_hoc_synth).intersection(hoc_ids_synth))

In [ ]:
len(set(llama_ids_hoc_synth).intersection(hoc_ids_synth[: len(llama_ids_hoc_synth)]))

In [ ]:
set(doc["agent"] for doc in synthesized_texts.find(projection=["agent"]))

In [ ]:
next(
    synthesized_texts.aggregate(
        [
            {"$match": {"type": "chunk", "agent": "gemma2:9b"}},
            # {"$limit": 1},
            {
                "$project": {
                    "_id": 1,
                    "chunks": 1,
                    "start_idx": "$synth_metadata.start_idx",
                    "end_idx": "$synth_metadata.end_idx",
                }
            },
            {
                "$lookup": {
                    "from": "transition_scores",
                    "as": "transition_scores",
                    "localField": "_id",
                    "foreignField": "document._synth_id.$id",
                    "pipeline": [
                        {"$match": {"document.type": "chunk", "model.name": "gpt2"}},
                        {
                            "$project": {
                                "_id": 1,
                                "document": 1,
                                "model": 1,
                                # "transition_scores": 1,
                                "metadata": 1,
                            }
                        },
                    ],
                }
            },
        ],
        # allowDiskUse=True,
    ),
)

In [ ]:
list(
    transition_scores.aggregate(
        [
            {
                "$lookup": {
                    "from": "collected_items",
                    "localField": "refs._ref_id.$id",
                    "foreignField": "_id",
                    "as": "source",
                }
            },
            {
                "$project": {
                    "transition_scores": 0,
                    "source.text": 0,
                    "source.chunks": 0,
                    "source.synthetization": 0,
                    # "_id": 1,
                    # "domain": 1,
                    # "features._id": 1,
                    # "features.model.name": 1,
                    # "features.pre_processor.type": 1,
                }
            },
            {"$unwind": {"path": "$source"}},
            {
                "$group": {
                    "_id": {
                        "$concat": [
                            "$source.domain",
                            " - ",
                            "$model.name",
                            " - ",
                            "$pre_processor.type",
                        ]
                    },
                    "total": {"$sum": 1},
                }
            },
        ]
    )
)

In [ ]:
list(
    transition_scores.aggregate(
        [
            {
                "$lookup": {
                    "from": "synthesized_texts",
                    "localField": "refs._ref_id.$id",
                    "foreignField": "_id",
                    "as": "source",
                }
            },
            {
                "$project": {
                    "transition_scores": 0,
                    "source.text": 0,
                    "source.chunks": 0,
                    "source.synthetization": 0,
                    # "_id": 1,
                    # "domain": 1,
                    # "features._id": 1,
                    # "features.model.name": 1,
                    # "features.pre_processor.type": 1,
                }
            },
            {"$unwind": {"path": "$source"}},
            {
                "$group": {
                    "_id": {
                        "$concat": [
                            "$source.domain",
                            " - ",
                            "$model.name",
                            " - ",
                            "$pre_processor.type",
                        ]
                    },
                    "total": {"$sum": 1},
                }
            },
        ]
    )
)

In [ ]:
sum(
    1
    for _ in test_swt.aggregate(
        [
            {
                "$lookup": {
                    "from": "synthesized_texts",
                    "localField": "refs._ref_id.$id",
                    "foreignField": "_id",
                    "as": "features",
                }
            },
            {"$match": {"features": {"$exists": True, "$ne": []}}},
            {
                "$project": {
                    "_id": 1,
                    "refs._ref_id.$id": 1,
                    "features._id": 1,
                    "features.id": 1,
                }
            },
        ]
    )
    # if doc["features"]
)

In [ ]:
sum(1 for _ in test_swt.find(projection={"_id": 1}))

In [ ]:
import re

re.findall("\d", "abc")